In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
batch_size = 100
num_classes = 10
epochs = 75
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#### LOAD AND TRANSFORM
# The output of torchvision datasets are PILImage images of range [0, 1].
data_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=data_transforms)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=data_transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
x_batch, y_batch = iter(train_loader).next()
print("Training set: {} samples - Max value: {} - Min value: {}".format(len(train_loader.dataset), 
                                                                        x_batch.max(), x_batch.min()))

Training set: 50000 samples - Max value: 1.0 - Min value: 0.0


In [5]:
x_batch, y_batch = iter(test_loader).next()
print("Test set: {} samples - Max value: {} - Min value: {}".format(len(test_loader.dataset), 
                                                                        x_batch.max(), x_batch.min()))

Test set: 10000 samples - Max value: 1.0 - Min value: 0.0


In [6]:
print("Example batch shape: {}".format(x_batch.shape))

Example batch shape: torch.Size([100, 3, 32, 32])


In [7]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """

    def __init__(self, sigma=0.1, is_relative_detach=True):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.noise = torch.tensor(0).to(device).float()

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.repeat(*x.size()).normal_() * scale
            x = x + sampled_noise
        return x 

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = self._CBGN(3, 32)
        self.block2 = self._CBGN(32, 64)
        self.block3 = self._CBGN(64, 128)
        self.block4 = self._CBGN(128, 256)
        self.block5 = self._CBGN(256, 512)
        # Flatten at forward!
        self.features = nn.Linear(512, 512)
        self.reluFeatures = nn.ReLU()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.block1(x)                         # 32x32 -> 16x16 @ 32
        out = self.block2(out)                       # 16x16 -> 8x8 @ 64
        out = self.block3(out)                       # 8x8 -> 4x4 @ 128
        out = self.block4(out)                       # 4x4 -> 2x2 @ 256
        out = self.block5(out)                       # 2x2 -> 1x1 @ 512
        out = out.view(out.size(0), -1)              # Flatten
        out = self.reluFeatures(self.features(out))  # 512 -> 512
        out = self.classifier(out)                   # 512 -> num_classes (10)
        return out

    # DEF A BLOCK Conv + BN + GN + MaxPool
    def _CBGN(self, in_channels, filters):
        layers = []
        layers += [nn.Conv2d(in_channels, filters, kernel_size=(3,3), padding=1)]
        layers += [nn.BatchNorm2d(filters)]
        layers += [GaussianNoise(0.3)]
        layers += [nn.ReLU()]
        layers += [nn.MaxPool2d(kernel_size=(2,2))]
        return nn.Sequential(*layers)


net = Net().to(device)
print(net)

Net(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GaussianNoise()
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GaussianNoise()
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GaussianNoise()
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block4): S

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, weight_decay=1e-6)

In [11]:
print("\n---- Start Training ----")
best_accuracy = -1
for epoch in range(epochs):

    # TRAIN THE NETWORK
    train_loss, train_correct = 0, 0
    net.train()
    for inputs, targets in train_loader:
        # data is a list of [inputs, labels]
        inputs, targets = inputs.to(device), targets.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        _, pred = outputs.max(1)  # get the index of the max log-probability
        train_correct += pred.eq(targets).sum().item()

        # print statistics
        train_loss += loss.item()
        
    train_loss /= len(train_loader.dataset)

    # TEST NETWORK
    net.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            test_loss += criterion(outputs, targets)
            _, pred = outputs.max(1)  # get the index of the max log-probability
            correct += pred.eq(targets).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    print("[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(epoch+1, train_loss, test_loss, 100. * train_correct / len(train_loader.dataset), test_accuracy))
    
    if test_accuracy>best_accuracy:
        best_accuracy = test_accuracy
    
print('Finished Training')
print("Best Test accuracy: {:.2f}".format(best_accuracy))


---- Start Training ----
[Epoch 1] Train Loss: 0.013797 - Test Loss: 0.012904 - Train Accuracy: 49.22% - Test Accuracy: 54.73%
[Epoch 2] Train Loss: 0.010004 - Test Loss: 0.009579 - Train Accuracy: 64.45% - Test Accuracy: 66.51%
[Epoch 3] Train Loss: 0.008480 - Test Loss: 0.012105 - Train Accuracy: 70.07% - Test Accuracy: 58.71%
[Epoch 4] Train Loss: 0.007582 - Test Loss: 0.009332 - Train Accuracy: 73.23% - Test Accuracy: 67.58%
[Epoch 5] Train Loss: 0.006883 - Test Loss: 0.014690 - Train Accuracy: 75.80% - Test Accuracy: 52.33%
[Epoch 6] Train Loss: 0.006376 - Test Loss: 0.008274 - Train Accuracy: 77.36% - Test Accuracy: 71.31%
[Epoch 7] Train Loss: 0.005888 - Test Loss: 0.007582 - Train Accuracy: 79.18% - Test Accuracy: 73.26%
[Epoch 8] Train Loss: 0.005493 - Test Loss: 0.008150 - Train Accuracy: 80.72% - Test Accuracy: 72.41%
[Epoch 9] Train Loss: 0.005199 - Test Loss: 0.007257 - Train Accuracy: 81.80% - Test Accuracy: 75.38%
[Epoch 10] Train Loss: 0.004849 - Test Loss: 0.007638 - 